# Econ 8210 Homework 1, Fall 2024
### Numerical Methods and Dynamic Programming
### Haosi Shen

In [1]:
# Housekeeping
import numpy as np
import pandas as pd

np.random.seed(42) 

## 1. Integration

Compute
\begin{equation*}
\int_{0}^{T} e^{-\rho t} u(1-e^{-\lambda t})\,dt
\end{equation*}
for $T=100$, $\rho = 0.04$, $\lambda = 0.02$, and $u(c)=-e^{-c}$ using **quadrature** (midpoint, Trapezoid, and Simpson rule) and Monte Carlo integration.

In [2]:
# Define Problem
T = 100
rho = 0.04
lambda_ = 0.02

def u(c):
    return -np.exp(-c)

def integrand(t):
    return np.exp(-rho * t) * u(1 - np.exp(-lambda_ * t))

# Number of intervals/draws
n_intervals = np.array([10, 100, 1000, 5000, 10000])

### Quadrature Integration

In [3]:
# Midpoint
def midpoint_quadrature(a, b, n):
    h = (b - a) / n
    total = 0
    for i in range(n):
        midpoint = a + (i + 0.5) * h
        total += integrand(midpoint)
    return h * total

vec_midpoint =  np.vectorize(midpoint_quadrature)
results_midpoint = vec_midpoint(0, T, n_intervals)

In [4]:
# Trapezoid
def trapezoid_quadrature(a, b, n):
    h = (b - a) / n
    total = 0.5 * (integrand(a) + integrand(b))
    for i in range(1, n):
        total += integrand(a + i * h)
    return h * total

vec_trapezoid =  np.vectorize(trapezoid_quadrature)
results_trapezoid = vec_trapezoid(0, T, n_intervals)

In [5]:
# Simpson's Rule
def simpsons_quadrature(a, b, n):
    if n % 2 == 1:
        n += 1  # ensure n is even
    h = (b - a) / n
    total = integrand(a) + integrand(b)
    for i in range(1, n, 2):
        total += 4 * integrand(a + i * h)
    for i in range(2, n, 2):
        total += 2 * integrand(a + i * h)
    return (h / 3) * total

vec_simpsons =  np.vectorize(simpsons_quadrature)
results_simpsons = vec_simpsons(0, T, n_intervals)

In [6]:
print("Number of Intervals:", n_intervals)
print("Integral using midpoint quadrature:", results_midpoint)
print("Integral using trapezoid quadrature:", results_trapezoid)
print("Integral using Simpson's rule:", results_simpsons)

Number of Intervals: [   10   100  1000  5000 10000]
Integral using midpoint quadrature: [-17.96441999 -18.20703949 -18.20950054 -18.20952441 -18.20952515]
Integral using trapezoid quadrature: [-18.70274754 -18.21449754 -18.20957513 -18.20952739 -18.2095259 ]
Integral using Simpson's rule: [-18.22464122 -18.20952704 -18.2095254  -18.2095254  -18.2095254 ]


### Monte Carlo Integration

In [7]:
def monteCarlo_integration(a, b, n):
    random_points = np.random.uniform(a, b, n)
    integral_estimate = (b - a) * np.mean([integrand(t) 
                                           for t in random_points])
    return integral_estimate

vec_monteCarlo =  np.vectorize(monteCarlo_integration)
results_monteCarlo = vec_monteCarlo(0, T, n_intervals)

In [8]:
results_integration = pd.DataFrame(np.stack((results_midpoint, results_trapezoid, 
                                             results_simpsons, results_monteCarlo)),
            columns = ['N = 10', 'N = 100', 'N = 1000', 'N = 5000', 'N = 10000'], 
            index = (['Midpoint', 'Trapezoid', 'Simpson\'s', 'Monte Carlo']))

display(results_integration)

,N = 10,N = 100,N = 1000,N = 5000,N = 10000
Midpoint,-17.964420,-18.207039,-18.209501,-18.209524,-18.209525
Trapezoid,-18.702748,-18.214498,-18.209575,-18.209527,-18.209526
Simpson's,-18.224641,-18.209527,-18.209525,-18.209525,-18.209525
Monte Carlo,-24.732456,-20.260672,-18.809211,-18.345481,-18.488059


## 2. Optimization
